In [122]:
import json 
import os
import logging 
import concurrent 
import asyncio 
import re 
import requests 

from collections import deque 
from itertools import chain 
# from datetime import date, datetime 

from typing import List 
from pprint import PrettyPrinter

from subgrounds import Subgrounds
from palettable.tableau import Tableau_20
from palettable.mycarta import Cube1_4, Cube1_8
from subgrounds.pagination import ShallowStrategy
from IPython.display import HTML, display
from web3 import Web3
from functools import partial 
from concurrent.futures import ThreadPoolExecutor

# apis / networking 
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from etherscan import Etherscan
from pycoingecko import CoinGeckoAPI

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss

# logging.basicConfig(level=logging.INFO)

# TODO: Figure out how this frax subgraph is useful
# https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxbp-subgraph/graphql

pp = PrettyPrinter().pprint

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [123]:
colors_24 = Tableau_20.hex_colors + Cube1_4.hex_colors
colors_28 = Tableau_20.hex_colors + Cube1_8.hex_colors

In [124]:
URL_INFURA = 'https://mainnet.infura.io/v3/856c3834f317452a82e25bb06e04de18'
w3 = Web3(Web3.HTTPProvider(URL_INFURA))
cg = CoinGeckoAPI()

In [220]:
def ddf(df):
    display(HTML(df.to_html()))
    
    
def remove_prefix(df: pd.DataFrame, prefix: str):
    # Remove a prefix from all columns 
    col_map = {}
    for i, c in enumerate(df.columns): 
        if c.startswith(prefix): 
            col_map[c] = c[len(prefix):]
    df = df.rename(columns=col_map) 
    return df 

def remove_prefixes(df: pd.DataFrame, prefixes: List[str]):
    for p in prefixes: 
        df = remove_prefix(df, p)
    return df 

def query_attrs(query, attrs):
    qattrs = []
    for a in attrs: 
        if '.' in a: 
            # nested
            v = None 
            for p in a.split('.'): 
                v = getattr(query, p) if v is None else getattr(v, p) 
            qattrs.append(v) 
        else: 
            # non-nested 
            qattrs.append(getattr(query, a)) 
    return sg.query_df(qattrs, pagination_strategy=ShallowStrategy) 

def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def df_cols_camel_to_snake(df):
    col_map = {c: camel_to_snake(c) for c in df.columns}
    df = df.rename(columns=col_map) 
    return df 

def df_cols_change_prefix(df, prefix_cur, prefix_new):
    col_map = {}
    for c in df.columns: 
        if c.startswith(prefix_cur): 
            col_map[c] = prefix_new + c[len(prefix_cur):]
    df = df.rename(columns=col_map) 
    return df 

In [221]:
URL_CURVE_POOLS = 'https://api.thegraph.com/subgraphs/name/convex-community/curve-pools'
URL_CURVE_DAO = 'https://api.thegraph.com/subgraphs/name/convex-community/curve-dao'

ADDRESS_FRXETHCRV_GAUGE = '0x2932a86df44fe8d2a706d8e9c5d51c24883423f5'.lower()
ADDRESS_CONVEX_VOTER_PROXY = '0x989aeb4d175e16225e39e87d0d97a3360524ad80'.lower()
ADDRESS_CURVE_POOL_FRXETH_ETH = '0xa1f8a6807c402e4a15ef4eba36528a3fed24e577'.lower()

In [127]:
# Instantiate the client with an endpoint.
sg = Subgrounds()

# votium_bribes = sg.load_subgraph(URL_VOTIUM) 
sg_curve_pools = sg.load_subgraph(URL_CURVE_POOLS)
sg_curve_dao = sg.load_subgraph(URL_CURVE_DAO) 

In [148]:
qattrs = ['id', 'provider', 'value']
# deposits into gauge 
df_deposits = query_attrs(
    sg_curve_dao.Query.gaugeDeposits(first=100000, where={'gauge': ADDRESS_FRXETHCRV_GAUGE}), qattrs
)
df_deposits = remove_prefix(df_deposits, 'gaugeDeposits_') 
# withdawals from gauge 
df_withdraws = query_attrs(
    sg_curve_dao.Query.gaugeWithdraws(first=100000, where={'gauge': ADDRESS_FRXETHCRV_GAUGE}), qattrs
)
df_withdraws = remove_prefix(df_withdraws, 'gaugeWithdraws_') 
df_withdraws.value *= -1
# Current deposited amount per address within gauge 
df_curve = pd.concat([df_deposits, df_withdraws])
df_curve = df_curve.groupby('provider')['value'].sum().reset_index()
df_curve = df_curve.loc[df_curve.value != 0].sort_values('value', ascending=False).reset_index(drop=True)
df_curve['platform'] = 'curve-gauge'
df_curve.value /= 1e18

In [149]:
# TODO: we ignore circulating for now since that's a very small sum 
frxethCRV_curve_gauge = df_curve.value.sum()
frxethCRV_convex = df_curve.loc[df_curve.provider == ADDRESS_CONVEX_VOTER_PROXY].value.sum()
print(f"Number of frxETHCRV deposited in curve gauge: {frxethCRV_curve_gauge:,.1f}")
print(f"Number of frxETHCRV staked on convex: {frxethCRV_convex:,.1f}")
print(f"Percent of frxETHCRV staked on convex: {frxethCRV_convex / frxethCRV_curve_gauge:%}")
df_curve.head()

Number of frxETHCRV deposited in curve gauge: 32,958.0
Number of frxETHCRV staked on convex: 30,383.6
Percent of frxETHCRV staked on convex: 92.188858%


,provider,value,platform
0,0x989aeb4d175e16225e39e87d0d97a3360524ad80,30383.619706,curve-gauge
1,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,2047.814958,curve-gauge
2,0x10e3085127c9bd92ab325f8d1f65cdcec2436149,199.879798,curve-gauge
3,0x29f227b10c58457d8837031c813216de82f9abaf,102.939595,curve-gauge
4,0x73e47e110dd251bd6449381724f2bb51c11b14bc,43.367171,curve-gauge


In [161]:
qattrs = ['id', 'amount', 'timestamp', 'user.address']
# deposits into the convex staking contract for curve lp tokens 
q_deposits = sg_curve_pools.Query.deposits(first=100000, where={'poolid_': {'swap': ADDRESS_CURVE_POOL_FRXETH_ETH}})
df_deposits = query_attrs(q_deposits, qattrs)
df_deposits = remove_prefix(df_deposits, 'deposits_') 
# withdrawals from the convex staking contract for curve lp tokens 
q_withdrawals = sg_curve_pools.Query.withdrawals(first=100000, where={'poolid_': {'swap': ADDRESS_CURVE_POOL_FRXETH_ETH}})
df_withdrawals = query_attrs(q_withdrawals, qattrs)
df_withdrawals = remove_prefix(df_withdrawals, 'withdrawals_') 
df_withdrawals.amount *= -1
# current deposited amount per account in convex staking contract for curve lp tokens 
df_convex = pd.concat([df_deposits, df_withdrawals])
df_convex.amount /= 1e18
df_convex['platform'] = 'convex-curve'
df_convex = df_convex.rename(columns={'user_address': 'account'})
df_convex.head()

,id,amount,timestamp,account,platform
0,0x007562ec43ffc63894d5b30e812bd65858605829090a...,1.019416,1669427219,0x0f903834187d37ff29dc2d607dd9fb50eb36b2b5,convex-curve
1,0x012a7730ff93a15624d92a94a9a8a5fa9ab8e2ff7b28...,6.79953,1668665207,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve
2,0x01d17a19490fc0bbc90d695152c9d60cbd071f40fba6...,199.860807,1668970739,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve
3,0x01d5801d769773d6fa0722668f1c8fcbf13f2e425c93...,10.0,1669056719,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve
4,0x03a493a4bcefca190cf3fe4c95d3564c9f262c43ec3f...,533.486109,1667486183,0x5a40ab806a038afe9c4ef413d36b6356a5e4d2c5,convex-curve


In [162]:
# These numbers should be really close but sometimes might differ due to subgraph indexing speed. 
convex_frxethcrv_1 = df_convex.amount.sum()
convex_frxethcrv_2 = df_curve.loc[df_curve.provider == ADDRESS_CONVEX_VOTER_PROXY]['value'].values.tolist()[0]
print(f"Convex frxETHCRV (convex staking): {convex_frxethcrv_1}")
print(f"Convex frxETHCRV (curve gauge): {convex_frxethcrv_2}")
np.testing.assert_almost_equal(convex_frxethcrv_1, convex_frxethcrv_2, 10)

Convex frxETHCRV (convex staking): 30383.61970627547
Convex frxETHCRV (curve gauge): 30383.61970627549


In [163]:
df_curve.head(1)

,provider,value,platform
0,0x989aeb4d175e16225e39e87d0d97a3360524ad80,30383.619706,curve-gauge


In [164]:
df_convex.head(1)

,id,amount,timestamp,account,platform
0,0x007562ec43ffc63894d5b30e812bd65858605829090a...,1.019416,1669427219,0x0f903834187d37ff29dc2d607dd9fb50eb36b2b5,convex-curve


In [165]:
df_curve_no_convex = df_curve.loc[df_curve.provider != ADDRESS_CONVEX_VOTER_PROXY]
df = (
    pd.concat([
        df_curve_no_convex.rename(columns={'provider': 'account', 'value': 'amount'})[['account', 'amount', 'platform']], 
        df_convex[['account', 'amount', 'platform']]
    ])
    .groupby(['account', 'platform'])['amount'].sum().reset_index()
    .sort_values('amount', ascending=False).reset_index(drop=True) 
)
# This total should match amount held by 0x2932a86df44fe8d2a706d8e9c5d51c24883423f5 (the frxETHCRV gauge deposit) 
# https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a#balances
print(df.amount.sum())
df.head()

32958.017308339935


,account,platform,amount
0,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve,26362.859298
1,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,curve-gauge,2047.814958
2,0x3cf54f3a1969be9916dad548f3c084331c4450b5,convex-curve,1559.840998
3,0xa1175a219dac539f2291377f77afd786d20e5882,convex-curve,612.773914
4,0xdb722dd612d35a12e718138e7296e6ca3531fb98,convex-curve,327.994192


In [166]:
address_convex_staking_wrapper_frax = '0x4659d5fF63A1E1EDD6D5DD9CC315e063c95947d0'
contract_convex_staking_wrapper_frax = w3.eth.contract(
    address_convex_staking_wrapper_frax, abi=eth.get_contract_abi(address)
)

In [137]:
deposit_logs = contract_convex_staking_wrapper_frax.events.Deposited.getLogs(fromBlock=0)
withdrawal_logs = contract_convex_staking_wrapper_frax.events.Withdrawn.getLogs(fromBlock=0)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:164: DeprecationWarning: abi.decode_single() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


In [207]:
df_stkcvxfrxethits = [{'account': d.args['_account'], 'amount': d.args['_amount'] / 1e18} for d in deposit_logs]
withdrawals = [{'account': d.args['_user'], 'amount': -d.args['_amount'] / 1e18} for d in withdrawal_logs]
df_stkcvxfrxeth = pd.DataFrame(deposits + withdrawals).groupby('account')['amount'].sum().reset_index()
df_stkcvxfrxeth = df_stkcvxfrxeth.sort_values('amount', ascending=False).reset_index(drop=True)
df_stkcvxfrxeth = df_stkcvxfrxeth.loc[df_stkcvxfrxeth.amount != 0] 
# Determine if this address represents a vault owned by some user 
abi_convex_staking_proxy = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"FEE_DENOMINATOR","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_rewardsAddress","type":"address"}],"name":"changeRewards","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"checkpointRewards","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"convexCurveBooster","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"convexDepositToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"crv","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"curveLpToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"cvx","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"earned","outputs":[{"internalType":"address[]","name":"token_addresses","type":"address[]"},{"internalType":"uint256[]","name":"total_earned","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRegistry","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"fxs","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_claim","type":"bool"},{"internalType":"address[]","name":"_rewardTokenList","type":"address[]"}],"name":"getReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_claim","type":"bool"}],"name":"getReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_owner","type":"address"},{"internalType":"address","name":"_stakingAddress","type":"address"},{"internalType":"address","name":"_stakingToken","type":"address"},{"internalType":"address","name":"_rewardsAddress","type":"address"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"_addl_liq","type":"uint256"}],"name":"lockAdditional","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"_addl_liq","type":"uint256"}],"name":"lockAdditionalConvexToken","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"_addl_liq","type":"uint256"}],"name":"lockAdditionalCurveLp","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"new_ending_ts","type":"uint256"}],"name":"lockLonger","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"poolRegistry","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"rewards","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_proxy","type":"address"}],"name":"setVeFXSProxy","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_liquidity","type":"uint256"},{"internalType":"uint256","name":"_secs","type":"uint256"}],"name":"stakeLocked","outputs":[{"internalType":"bytes32","name":"kek_id","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_liquidity","type":"uint256"},{"internalType":"uint256","name":"_secs","type":"uint256"}],"name":"stakeLockedConvexToken","outputs":[{"internalType":"bytes32","name":"kek_id","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_liquidity","type":"uint256"},{"internalType":"uint256","name":"_secs","type":"uint256"}],"name":"stakeLockedCurveLp","outputs":[{"internalType":"bytes32","name":"kek_id","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"stakingAddress","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"stakingToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"usingProxy","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"vaultType","outputs":[{"internalType":"enum IProxyVault.VaultType","name":"","type":"uint8"}],"stateMutability":"pure","type":"function"},{"inputs":[],"name":"vaultVersion","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[],"name":"vefxsProxy","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"}],"name":"withdrawLocked","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"}],"name":"withdrawLockedAndUnwrap","outputs":[],"stateMutability":"nonpayable","type":"function"}]'
vault_owners = {}
for a in df_stkcvxfrxeth.account.unique(): 
    is_vault = True if w3.eth.getCode(a) else False
    if is_vault: 
        contract = w3.eth.contract(a, abi=abi_convex_staking_proxy)
        owner = contract.functions.owner().call()
        vault_owners[a] = owner
# Update table with vault info 
df_stkcvxfrxeth['vault_address'] = df_stkcvxfrxeth.account.apply(lambda a: a if a in vault_owners else None)
mask_vault = ~df_stkcvxfrxeth.vault_address.isna()
df_stkcvxfrxeth.loc[mask_vault, 'account'] = df_stkcvxfrxeth.loc[mask_vault].account.apply(lambda a: vault_owners[a])
assert all(df_stkcvxfrxeth.loc[mask_vault].account != df_stkcvxfrxeth.loc[mask_vault].vault_address)
# Add in platform information 
df_stkcvxfrxeth['platform'] = 'convex-frax-no-vault'
df_stkcvxfrxeth.loc[~df_stkcvxfrxeth.vault_address.isna(), 'platform'] = 'convex-frax-vault'
df_stkcvxfrxeth.tail()

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/web3/method.py:215: DeprecationWarning: getCode is deprecated in favor of get_code
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:87: DeprecationWarning: abi.encode_abi() and abi.encode_abi_packed() are deprecated and will be removed in version 4.0.0 in favor of abi.encode() and abi.encode_packed(), respectively
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


,account,amount,vault_address
163,0xFf6A37C2fb94C3fB47CbeAB8053722084C546a32,0.452667,0xbaaf59e5572Ba0432D33Cd7bC09C1FDAA6393e0f
164,0x8d64898e2f89e3Ca1fcD5684882812EB4960c985,0.199905,0xB5f32EbB40b7dE0fB73E1Da427B1047c8F5C3ee0
165,0x5180db0237291A6449DdA9ed33aD90a38787621c,0.100000,None
166,0x439Adfe7CD81d8f471C16528Dbc034F87Aceb599,0.024000,0xd1a2971C3E54A476D348a073e88b45EBF2C59e81
167,0x439Adfe7CD81d8f471C16528Dbc034F87Aceb599,0.020000,None


In [211]:
# These numbers should be really close but sometimes might differ due to subgraph indexing speed. 
staked_convex_frxethcrv_1 = df_stkcvxfrxeth.amount.sum()
staked_convex_frxethcrv_2 = df.loc[df.account == address_convex_staking_wrapper_frax.lower()]['amount'].values.tolist()[0]
print(f"Convex stkfrxETHCRV (convex curve staking): {staked_convex_frxethcrv_1}")
print(f"Convex stkfrxETHCRV (convex frax staking): {staked_convex_frxethcrv_2}")
np.testing.assert_almost_equal(staked_convex_frxethcrv_1, staked_convex_frxethcrv_2, 10)

Convex stkfrxETHCRV (convex curve staking): 26362.85929812243
Convex stkfrxETHCRV (convex frax staking): 26362.85929812242


In [212]:
df.head(1)

,account,platform,amount
0,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve,26362.859298


In [213]:
df_stkcvxfrxeth.head(1)

,account,amount,vault_address,platform
0,0x8D8B9c79196f32161BcB2A9728D274B3b45eB9AF,4968.188152,0x1Cf851733F952c7Bc2aE375e370528d162d066e0,convex-frax-vault


In [226]:
df_no_stkcvxfrxeth = df.loc[df.account != address_convex_staking_wrapper_frax.lower()]
assert len(df) == 1 + len(df_no_stkcvxfrxeth)
df_final = (
    pd.concat([df_no_stkcvxfrxeth, df_stkcvxfrxeth[['account', 'platform', 'amount']]])
    .groupby('account')['amount'].sum().reset_index()
)
# This amount should match the amount held in the gauge for this curve pool. Check this value here: 
# https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a#balances
print(df_final.amount.sum())
df_final = df_final.sort_values('amount', ascending=False).reset_index(drop=True)
df_final.head()

32958.01730833995


,account,amount
0,0x8D8B9c79196f32161BcB2A9728D274B3b45eB9AF,4968.188152
1,0x8306300ffd616049FD7e4b0354a64Da835c1A81C,4325.813192
2,0xD07993c6cb9692a71522Baf970A31069034dF2B0,3170.969205
3,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,2047.814958
4,0x3cf54f3a1969be9916dad548f3c084331c4450b5,1559.840998
